In [1]:
import sys
path = "/home/someone/stage_jonathan/lightning-sam/lightning-sam/l_sam_env/lib/python3.10/site-packages"

if path not in sys.path: sys.path.append(path)

path2 = "/usr/lib/python3/dist-packages"

if path2 not in sys.path: sys.path.append(path2)

In [3]:
import pandas as pd
import os

In [11]:
file_path = "/mnt/data/dataverse_files/"
os.listdir(file_path)

['HAM10000_images_part_2',
 'ISIC2018_Task3_Test_GroundTruth.csv',
 'HAM10000_segmentations_lesion_tschandl',
 'ISIC2018_Task3_Test_NatureMedicine_AI_Interaction_Benefit.csv',
 'HAM10000_metadata',
 'ISIC2018_Task3_Test_Images',
 'HAM10000_images_part_1']

In [17]:
GT = pd.read_csv(os.path.join(file_path,"ISIC2018_Task3_Test_GroundTruth.csv"))
GT.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAMTEST_0000000,ISIC_0034524,nv,follow_up,40.0,female,back,vidir_molemax
1,HAMTEST_0000001,ISIC_0034525,nv,histo,70.0,male,abdomen,rosendahl
2,HAMTEST_0000002,ISIC_0034526,bkl,histo,70.0,male,back,rosendahl
3,HAMTEST_0000003,ISIC_0034527,nv,histo,35.0,male,trunk,vienna_dias
4,HAMTEST_0000004,ISIC_0034528,nv,follow_up,75.0,female,trunk,vidir_molemax


In [78]:
MD = pd.read_csv(os.path.join(file_path,"HAM10000_metadata"))
MD["image_name"]=MD["image_id"]+".jpg"
MD["mask_name"] = MD["image_id"]+"_segmentation.png"
MD.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image_name,mask_name
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0027419.jpg,ISIC_0027419_segmentation.png
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0025030.jpg,ISIC_0025030_segmentation.png
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0026769.jpg,ISIC_0026769_segmentation.png
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0025661.jpg,ISIC_0025661_segmentation.png
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,ISIC_0031633.jpg,ISIC_0031633_segmentation.png


In [38]:
print(len(MD["lesion_id"].unique()))
MD.info()

7470
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   dataset       10015 non-null  object 
 8   image_name    10015 non-null  object 
 9   mask_name     10015 non-null  object 
dtypes: float64(1), object(9)
memory usage: 782.5+ KB


In [39]:
print(MD["dx"].unique())

['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']


Actinic keratoses and intraepithelial carcinoma / Bowen's disease (**akiec**), basal cell carcinoma (**bcc**), benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, **bkl**), dermatofibroma (**df**), melanoma (**mel**), melanocytic nevi (**nv**) and vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, **vasc**).

In [42]:
temp = list(MD["dx"])
for label in MD["dx"].unique():
    print(label,temp.count(label))
    
del temp

bkl 1099
nv 6705
df 115
mel 1113
vasc 142
bcc 514
akiec 327


In [149]:
# class_id generation
dict_cat = {label:int(i) for i,label in enumerate(MD["dx"].unique())}
dict_cat

{'bkl': 0, 'nv': 1, 'df': 2, 'mel': 3, 'vasc': 4, 'bcc': 5, 'akiec': 6}

In [44]:
import cv2

In [27]:
print(len(MD))
print(len(GT))

10015
1512


In [15]:
list_images = os.listdir(file_path+"HAM10000_images_part_1")+os.listdir(file_path+"HAM10000_images_part_2")
print(len(list_images))

10015


In [25]:
all(image in list(MD["image_name"]) for image in list_images)

True

In [45]:
mask_path = "/mnt/data/dataverse_files/" + 2*"HAM10000_segmentations_lesion_tschandl/"
all(mask in list(MD["mask_name"]) for mask in os.listdir(mask_path))

True

In [79]:
MD.set_index("mask_name",inplace=True)

In [80]:
MD

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image_name
mask_name,,,,,,,,,
ISIC_0027419_segmentation.png,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0027419.jpg
ISIC_0025030_segmentation.png,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0025030.jpg
ISIC_0026769_segmentation.png,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0026769.jpg
ISIC_0025661_segmentation.png,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,ISIC_0025661.jpg
ISIC_0031633_segmentation.png,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,ISIC_0031633.jpg
...,...,...,...,...,...,...,...,...,...
ISIC_0033084_segmentation.png,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,ISIC_0033084.jpg
ISIC_0033550_segmentation.png,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,ISIC_0033550.jpg
ISIC_0033536_segmentation.png,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,ISIC_0033536.jpg


In [220]:
from itertools import groupby

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [224]:
import cv2
import numpy as np
from pycocotools import mask as maskUtils

from tqdm import tqdm

dict_mask = {mask_name:None for mask_name in os.listdir(mask_path)}

for mask_name in tqdm(dict_mask.keys()):
    mask = cv2.imread(mask_path+mask_name)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    area_ = float(np.sum(mask//255))

    coords = np.where(mask==255)
    
    x, xp, y, yp = np.min(coords[1]), np.max(coords[1]), np.min(coords[0]), np.max(coords[0])

    x,y,w,h = x,y,xp-x,yp-y

    seg = binary_mask_to_rle(mask)
    
    
    
    
    
    
    id_ = (mask_name.split("_")[1])
    
    cat_ = MD.loc[mask_name]["dx"]

    bbox_ = list(map(int,[x,y,w,h]))
    
    
    
    dict_mask[mask_name] = {"id":int(id_),
                            "image_id":(id_),
                            "file_name":id_+".jpg",
                            "category_id":dict_cat[cat_],
                            "segmentation":seg,
                            "area":area_,
                            "bbox":bbox_
                           }
    
    
    

print("Anot done")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10015/10015 [06:12<00:00, 26.87it/s]

Anot done


In [225]:
print(dict_mask.get(list(dict_mask.keys())[0]))

{'id': 32749, 'image_id': '0032749', 'file_name': '0032749.jpg', 'category_id': 1, 'segmentation': {'counts': [20851, 4, 433, 23, 421, 30, 418, 39, 408, 52, 396, 59, 389, 63, 384, 75, 374, 78, 371, 81, 368, 85, 364, 90, 359, 95, 355, 99, 350, 102, 347, 104, 345, 108, 342, 109, 341, 110, 339, 111, 339, 113, 335, 116, 334, 118, 330, 121, 327, 125, 323, 128, 318, 133, 315, 137, 312, 139, 309, 142, 306, 144, 305, 146, 303, 147, 302, 149, 300, 151, 298, 153, 295, 157, 292, 160, 289, 163, 286, 167, 283, 168, 281, 171, 278, 173, 276, 175, 274, 176, 273, 178, 271, 180, 269, 181, 268, 183, 266, 185, 265, 185, 264, 187, 262, 189, 260, 190, 260, 190, 259, 193, 256, 195, 254, 196, 254, 197, 252, 199, 250, 201, 248, 203, 246, 205, 244, 207, 242, 210, 240, 211, 238, 213, 236, 217, 232, 220, 230, 222, 228, 223, 226, 226, 224, 227, 222, 229, 221, 231, 219, 232, 218, 233, 216, 235, 215, 236, 213, 238, 212, 239, 211, 240, 210, 240, 210, 241, 208, 243, 207, 243, 207, 243, 207, 243, 206, 244, 206, 245, 20

In [226]:
# categories

cat = [
    {"id":dict_cat[key],
    "name":key}
    for key in dict_cat.keys()
]
cat

[{'id': 0, 'name': 'bkl'},
 {'id': 1, 'name': 'nv'},
 {'id': 2, 'name': 'df'},
 {'id': 3, 'name': 'mel'},
 {'id': 4, 'name': 'vasc'},
 {'id': 5, 'name': 'bcc'},
 {'id': 6, 'name': 'akiec'}]

In [227]:
annotations_list = list(dict_mask.values())
print(size := len(annotations_list))

10015


In [228]:
import random as rd
rd.shuffle(annotations_list)

train_annot = annotations_list[:int(0.8*size)]
val_annot   = annotations_list[int(0.8*size):]

In [229]:
any(val in train_annot for val in val_annot)

False

In [230]:
import json
with open(file_path+"train_anot.json","w") as file:
    json.dump(
        {
            "categories":cat,
            "annotations":train_annot
        },
        file,
        indent=4
    )

In [231]:
with open(file_path+"val_anot.json","w") as file:
    json.dump(
        {
            "categories":cat,
            "annotations":val_annot
        },
        file,
        indent=4
    )

In [ ]:
pwd

In [253]:
cd /mnt/data/dataverse_files/HAM10000_train/

/mnt/data/dataverse_files/HAM10000_train


In [255]:
val_names = [an["file_name"] for an in val_annot]

for file in val_names:
    FILE_NAME = "ISIC_"+file
    !mv $FILE_NAME "/mnt/data/dataverse_files/HAM10000_val/"


In [258]:
for folder in os.listdir():
    print(folder,le)

/mnt/data/dataverse_files
